In [28]:
import numpy as np
from scipy.special import ndtr
from scipy.stats import norm, ncx2
from scipy.optimize import minimize
from numpy.polynomial.hermite import hermfit, hermval, hermder
import copy
import fixed_income_derivatives as fid
import matplotlib.pyplot as plt


#### Spot rates to ZCB


In [14]:
def spot_rates_to_zcb(T,spot_rate):
    M = len(T)
    p = np.zeros([M])
    for i in range(0,M):
        if T[i] < 1e-8:
            p[i] = 1
        else:
            p[i] = np.exp(-spot_rate[i]*T[i])
    return p

#Example:
T = [1/4, 2/4, 3/4, 4/4]
spot_rate = [ 1.42669978,  0.4462871 ,  0.14048069, -0.        ]

print(spot_rates_to_zcb(T,spot_rate))

[0.7 0.8 0.9 1. ]


Input:
\begin{equation*}
\begin{aligned}
T &= \text{time to maturity} \\
\text{spot\_rate}&=\text{Spot rate}
\end{aligned}
\end{equation*}

Output:
\begin{equation*}
\begin{aligned}
p &= \text{ZCB price} 
\end{aligned}
\end{equation*}

Formula:
\begin{equation*}
\begin{aligned}
p(t,T)=\exp(-R(t,T)\cdot(T-t))
\end{aligned}
\end{equation*}


#### ZCB to spot rates

In [13]:
def zcb_to_spot_rates(T,p):
    M = len(T)
    R = np.zeros([M])
    for i in range(0,M):
        if T[i] < 1e-8:
            R[i] = 0
        else:
            R[i] = -np.log(p[i])/T[i]
    return R

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

print(zcb_to_spot_rates(T,p))

[ 1.42669978  0.4462871   0.14048069 -0.        ]


Input:
\begin{equation*}
\begin{aligned}
T &= \text{time to maturity} \\
p &= \text{ZCB price} 
\end{aligned}
\end{equation*}

Output:
\begin{equation*}
\begin{aligned}
R &= \text{spot rate} 
\end{aligned}
\end{equation*}

Formula:
\begin{equation*}
\begin{aligned}
R(t,T) &= -\frac{\log p(t,T)}{(T-t)}
\end{aligned}
\end{equation*}


#### ZCB to forward rates

In [22]:
def zcb_to_forward_rates(T,p,horizon = 1):
    # horizon = 0 corresponds to instantaneous forward rates
    M = len(T)
    f = np.zeros([M])
    if horizon == 0:
        f[0] = (np.log(p[0])-np.log(p[1]))/(T[1]-T[0])
        f[-1] = (np.log(p[-2])-np.log(p[-1]))/(T[-1]-T[-2])
        m = 1
        while m < M - 1.5: #1.5 is abritrary and might as well be 1.0001, as far as I understand...
            f[m] = (np.log(p[m-1])-np.log(p[m+1]))/(T[m+1]-T[m-1])
            m += 1
    elif 0 < horizon:
        m = horizon
        while m < M - 0.5:
            f[m] = (np.log(p[m-horizon])-np.log(p[m]))/(T[m]-T[m-horizon])
            m += 1
    return f

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

print(zcb_to_forward_rates(T,p, horizon=1))
print(zcb_to_forward_rates(T,p, horizon=0))

[ 0.         -0.53412557 -0.47113214 -0.42144206]
[-0.53412557 -0.50262886 -0.4462871  -0.42144206]


Input:
\begin{equation*}
\begin{aligned}
T &= \text{time to maturity} \\
p &= \text{ZCB price} \\
horizon&=\text{start date of loan}
\end{aligned}
\end{equation*}

Output:
\begin{equation*}
\begin{aligned}
f=\text{forward rates of return}
\end{aligned}
\end{equation*}

Formula:
\begin{equation*}
\begin{aligned}
R(t;S,T)=-\frac{\log p(t,T)-\log p(t,S)}{(T-S)}
\end{aligned}
\end{equation*}


#### ZCB to forward LIBOR rates

In [24]:
def zcb_to_forward_LIBOR_rates(T,p,horizon = 1):
    M = len(T)
    f = np.zeros([M])
    i = horizon
    #ERROR: Need to include what happens when horizon = 0
    while i < M - 0.5:
        f[i] = (p[i-horizon]-p[i])/(p[i]*(T[i]-T[i-horizon]))
        i += 1
    return f

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

print(zcb_to_forward_LIBOR_rates(T,p, horizon=1))


[ 0.         -0.5        -0.44444444 -0.4       ]


Input:
\begin{equation*}
\begin{aligned}
T &= \text{time to maturity} \\
p &= \text{ZCB price} \\
horizon&=\text{start date of loan}
\end{aligned}
\end{equation*}

Output:
\begin{equation*}
\begin{aligned}
f=\text{forward rates of return}
\end{aligned}
\end{equation*}

Formula (unsure):
\begin{equation*}
\begin{aligned}
\text{Forward LIBOR Rate} = \frac{p(t, S) - p(t, T)}{p(t, T) \cdot (T - S)}
\end{aligned}
\end{equation*}


#### Utility function used in many of the following functions:

In [85]:
def value_in_list_returns_I_idx(value,list):
    output = False, None
    for i, item in enumerate(list):
        if abs(value-item) < 1e-6:
            output = True, i
            break
    return output

#Example:
example_list = [1/8, 2/8, 3/8, 4/8, 5/8, 6/8, 7/8, 8/8]
example_value = [2/8, 4/8, 6/8, 8/8]

for i in range(0,len(example_value)):
    I_fix, idx_fix = value_in_list_returns_I_idx(example_value[i], example_list)
    if I_fix is True:
        print(example_list[idx_fix])

0.25
0.5
0.75
1.0


In essence, the function only returns values that are in both lists.

#### ZCB to accrual factor (also known as present value of a basis point)

In [127]:
def zcb_to_accrual_factor(T_n,T_N,fixed_freq,T,p):
    if fixed_freq == "quarterly":
        p_fix = np.zeros([int(T_N-T_n)*4])
        T_fix = np.array([T_n + i*0.25 for i in range(1,int(T_N-T_n)*4 + 1)])
    elif fixed_freq == "semiannual":
        p_fix = np.zeros([int(T_N-T_n)*2])
        T_fix = np.array([T_n + i*0.5 for i in range(1,int(T_N-T_n)*2 + 1)])
    elif fixed_freq == "annual":
        p_fix = np.zeros([int(T_N-T_n)])
        T_fix = np.array([T_n + i*0.25 for i in range(1,int(T_N-T_n) + 1)])
    S = 0
    for i in range(0,len(T_fix)):
        I_fix, idx_fix = value_in_list_returns_I_idx(T_fix[i],T)
        if I_fix is True:
            T_fix[i] = T[idx_fix]
            p_fix[i] = p[idx_fix]
            if i == 0:
                S += T_fix[i]*p_fix[i]
            else:
                S += (T_fix[i]-T_fix[i-1])*p_fix[i]


    return S

#Example:
T = [1/4, 2/4, 3/4, 4/4]
p = [0.7, 0.8, 0.9, 1]

S = zcb_to_accrual_factor(0,1,"quarterly",T,p)

print(S)


0.85


Input:
\begin{equation*}
\begin{aligned}
T_n &= \text{Start time of the accrual period} \\
T_N &= \text{End time of the accrual period} \\
\text{fixed\_freq} &= \text{Frequency of accrual} \\
T &= \text{time to maturity} \\
p &= \text{ZCB price}
\end{aligned}
\end{equation*}

Output:
\begin{equation*}
\begin{aligned}
S=\text{Accrual factor}
\end{aligned}
\end{equation*}

Formula:
\begin{equation*}
\begin{aligned}
S_{n}^{k}(t) &=\sum_{i=n+1}^{k}\alpha_{i}p(t,T_{i}) \\
\alpha_{i} &=T_{i}-T_{i-1}
\end{aligned}
\end{equation*}

where $n$ is the start period and $k$ is the end period.


#### ZCB to par swap rate

In [129]:
def zcb_to_par_swap_rate(T_n,T_N,fixed_freq,T,p):
    I_n, idx_n = value_in_list_returns_I_idx(T_n,T)
    I_N, idx_N = value_in_list_returns_I_idx(T_N,T)
    D = p[idx_n]-p[idx_N]
    S = zcb_to_accrual_factor(T_n,T_N,fixed_freq,T,p)
    R = D/S
    return R

#Example:
T = [0, 1/4, 2/4, 3/4, 4/4]
p = [0.5, 0.7, 0.8, 0.9, 1]

zcb_to_par_swap_rate(0,1,"quarterly",T,p)

-0.5882352941176471

Input:
\begin{equation*}
\begin{aligned}
T_n &= \text{Start time of the accrual period} \\
T_N &= \text{End time of the accrual period} \\
\text{fixed\_freq} &= \text{Frequency of accrual} \\
T &= \text{time to maturity} \\
p &= \text{ZCB price}
\end{aligned}
\end{equation*}

Output:
\begin{equation*}
\begin{aligned}
R=\text{Par swap rate}
\end{aligned}
\end{equation*}

Formula:
\begin{equation*}
\begin{aligned}
R_{n}^{N}(t)=\frac{p_{n}(t)-p_{N}(t)}{\sum_{i=n+1}^{N}\alpha_{i}p_{i}(t)}
\end{aligned}
\end{equation*}

where $n$ is the start period and $N$ is the end period. Note that the denominator is the accrual rate.


#### ZCB price for vasicek

In [144]:
def zcb_price_vasicek(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        B = (1/a)*(1-np.exp(-a*tau))
        A = (B-tau)*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
        p = np.exp(A-r0*B)
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        p = np.zeros([M])
        for i in range(0,M):
            B = (1/a)*(1-np.exp(-a*tau[i]))
            A = (B-tau[i])*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
            #ERROR: Why is it not B**2????
            p[i] = np.exp(A-r0*B)
    else:
        print(f"tau not a recognized type")
        p = False
    return p

#Example:
r0, a, b, sigma_vasicek = 0.025, 0.5, 0.025, 0.02
alpha = 0.25
T_max = 10

M = int(round(T_max/alpha))

T = np.array([i*alpha for i in range(0,M+1)])

print(zcb_price_vasicek(r0, a, b, sigma_vasicek, T))

[1.         0.99336228 0.98611486 0.97834796 0.97014137 0.96156554
 0.9526825  0.94354674 0.93420609 0.9247024  0.91507227 0.90534762
 0.89555625 0.88572234 0.87586689 0.86600808 0.85616164 0.84634112
 0.83655822 0.82682295 0.81714389 0.80752836 0.79798256 0.78851175
 0.77912033 0.76981198 0.76058975 0.75145611 0.74241308 0.73346224
 0.72460482 0.71584171 0.70717356 0.69860077 0.69012352 0.68174184
 0.67345559 0.6652645  0.6571682  0.64916622 0.64125799]


Input:
\begin{equation*}
\begin{aligned}
r_0 &= \text{Initial short rate} \\
a &= \text{Speed of reversion} \\
b &= \text{Long-term mean level} \\
\sigma &= \text{Volatility} \\
\tau &= \text{Time to maturity}
\end{aligned}
\end{equation*}

Output:
\begin{equation*}
p = \text{Price of the ZCB}
\end{equation*}

Formula:
\begin{equation*}
p(t, T) = e^{A(t,T)-B(t,T)r(t)}
\end{equation*}
where
\begin{equation*}
\begin{aligned}
A(t,T)&=\frac{\left[B(t,T)-(T-t)\right]\left(ab-\frac{1}{2}\sigma^{2}\right)}{a^{2}}-\frac{\sigma^{2}B^{2}(t,T)}{4a}\\
B(t, T) &= \frac{1}{a}\left[1 - e^{-a(T-t)}\right]
\end{aligned}
\end{equation*}

#### Spot rate for vasicek

In [142]:
def spot_rate_vasicek(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        B = (1/a)*(1-np.exp(-a*tau))
        A = (B-tau)*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
        if tau < 1e-6:
            r = 0
        elif tau >= 1e-6:
            r = (-A+r0*B)/tau
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        r = np.zeros([M])
        for i in range(0,M):
            B = (1/a)*(1-np.exp(-a*tau[i]))
            A = (B-tau[i])*(a*b-0.5*sigma**2)/(a**2)-(sigma**2*B)/(4*a)
            if tau[i] < 1e-6:
                r[i] = 0
            else:
                r[i] = (-A+r0*B)/tau[i]
    else:
        print(f"tau not a recognized type")
        r = False
    return r

#Example:
r0, a, b, sigma_vasicek = 0.025, 0.5, 0.025, 0.02
alpha = 0.25
T_max = 10

M = int(round(T_max/alpha))

T = np.array([i*alpha for i in range(0,M+1)])

print(spot_rate_vasicek(r0, a, b, sigma_vasicek, T))

[0.         0.02663941 0.02796488 0.02918651 0.03031347 0.03135404
 0.03231573 0.03320536 0.03402911 0.03479259 0.03550089 0.03615865
 0.03677008 0.037339   0.0378689  0.03836294 0.03882402 0.03925477
 0.03965759 0.04003467 0.04038802 0.04071945 0.04103064 0.04132313
 0.0415983  0.04185744 0.04210172 0.04233222 0.04254992 0.04275575
 0.04295051 0.04313499 0.04330989 0.04347586 0.04363349 0.04378334
 0.04392591 0.04406168 0.04419108 0.04431451 0.04443234]


Input: Same as above

Output: Spot rate

Formula:

\begin{equation*}
R(t,T)=\frac{-A(t,T)+rB(t,T)}{T-t}
\end{equation*}

where $A$ and $B$ are the same as above.

#### Forward rate vasicek

In [145]:
def forward_rate_vasicek(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        B = (1/a)*(1-np.exp(-a*tau))
        B_T = np.exp(-a*tau)
        if tau < 1e-6:
            f = 0
        elif tau >= 1e-6:
            f = (1-B_T)*(a*b-0.5*sigma**2)/(a**2) + (sigma**2*B*B_T)/(2*a) + r0*B_T
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        f = np.zeros([M])
        for i in range(0,M):
            B = (1/a)*(1-np.exp(-a*tau[i]))
            B_T = np.exp(-a*tau[i])
            if tau[i] < 1e-6:
                f[i] = 0
            else:
                f[i] = (1-B_T)*(a*b-0.5*sigma**2)/(a**2) + (sigma**2*B*B_T)/(2*a) + r0*B_T
    else:
        print(f"tau not a recognized type")
        f = False
    return f

#Example:
r0, a, b, sigma_vasicek = 0.025, 0.5, 0.025, 0.02
alpha = 0.25
T_max = 10

M = int(round(T_max/alpha))

T = np.array([i*alpha for i in range(0,M+1)])

print(forward_rate_vasicek(r0, a, b, sigma_vasicek, T))

[0.         0.02792653 0.03049084 0.03273954 0.03471288 0.03644568
 0.03796812 0.03930641 0.04048335 0.04151881 0.04243012 0.04323241
 0.04393892 0.04456125 0.04510953 0.04559268 0.0460185  0.04639386
 0.04672477 0.04701654 0.04727382 0.04750071 0.04770082 0.04787732
 0.048033   0.04817033 0.04829148 0.04839836 0.04849265 0.04857584
 0.04864924 0.048714   0.04877115 0.04882157 0.04886605 0.04890531
 0.04893995 0.04897052 0.04899749 0.04902129 0.0490423 ]


Input: Same as above

Output: Forward rate

Formula:

\begin{equation*}
f(t,T)=\frac{(1-B_{T})(ab-\frac{1}{2}\sigma^{2})}{a^{2}}+\frac{\sigma^{2}BB_{T}}{2a}+rB_{t}
\end{equation*}

where $A$ and $B$ are the same as above and $B_{t}=\exp(-a\cdot(T-t))$



#### Confidence intervals for vasicek

In [35]:
def ci_vasicek(r0,a,b,sigma,T,size_ci,method = "two_sided"):
    if type(T) == int or type(T) == float:
        if method == "lower":
            z = norm.ppf(size_ci,0,1)
            if T < 1e-6:
                lb, ub = np.nan, np.nan
            else:
                mean = r0*np.exp(-a*T) + b/a*(1-np.exp(-a*T))
                std = np.sqrt(sigma**2/(2*a)*(1-np.exp(-2*a*T)))
                lb, ub = mean - z*std, np.inf
        elif method == "upper":
            z = norm.ppf(size_ci,0,1)
            if T < 1e-6:
                lb, ub = np.nan, np.nan
            else:
                mean = r0*np.exp(-a*T) + b/a*(1-np.exp(-a*T))
                std = np.sqrt(sigma**2/(2*a)*(1-np.exp(-2*a*T)))
                lb, ub = -np.inf, mean + z*std
        elif method == "two_sided":
            z = norm.ppf(size_ci + 0.5*(1-size_ci),0,1)
            if T < 1e-6:
                lb, ub = np.nan, np.nan
            else:
                mean = r0*np.exp(-a*T) + b/a*(1-np.exp(-a*T))
                std = np.sqrt(sigma**2/(2*a)*(1-np.exp(-2*a*T)))
                lb, ub = mean - z*std, mean + z*std
        print(f"method: {method}, z: {z}")
    elif type(T) == tuple or type(T) == list or type(T) == np.ndarray:
        N = len(T)
        lb, ub = np.zeros([N]), np.zeros([N])
        if method == "lower":
            z = norm.ppf(size_ci,0,1)
            for i in range(0,N):
                if T[i] < 1e-6:
                    lb[i], ub[i] = np.nan, np.nan
                else:
                    mean = r0*np.exp(-a*T[i]) + b/a*(1-np.exp(-a*T[i]))
                    std = np.sqrt(sigma**2/(2*a)*(1-np.exp(-2*a*T[i])))
                    lb[i], ub[i] = mean - z*std, np.inf
        elif method == "upper":
            z = norm.ppf(size_ci,0,1)
            for i in range(0,N):
                if T[i] < 1e-6:
                    lb[i], ub[i] = np.nan, np.nan
                else:
                    mean = r0*np.exp(-a*T[i]) + b/a*(1-np.exp(-a*T[i]))
                    std = np.sqrt(sigma**2/(2*a)*(1-np.exp(-2*a*T[i])))
                    lb[i], ub[i] = -np.inf, mean + z*std
        elif method == "two_sided":
            z = norm.ppf(size_ci + 0.5*(1-size_ci),0,1)
            for i in range(0,N):
                if T[i] < 1e-6:
                    lb[i], ub[i] = np.nan, np.nan
                else:
                    mean = r0*np.exp(-a*T[i]) + b/a*(1-np.exp(-a*T[i]))
                    std = np.sqrt(sigma**2/(2*a)*(1-np.exp(-2*a*T[i])))
                    lb[i], ub[i] = mean - z*std, mean + z*std
    else:
        print(f"tau not a recognized type")
        lb,ub = False, False
    return lb, ub, 


#Example:
r0, a, b, sigma_vasicek = 0.025, 0.5, 0.025, 0.02
alpha = 0.25
T_max = 10

M = int(round(T_max/alpha))

T = np.array([i*alpha for i in range(0,M+1)])

lb, ub = ci_vasicek(r0, a, b, sigma_vasicek, T, 0.95, method = "two_sided")

print(lb)
print(ub)


[       nan 0.00950144 0.0059414  0.00434405 0.00367095 0.00350736
 0.0036405  0.00394749 0.00435264 0.0048077  0.00528138 0.00575339
 0.00621073 0.0066454  0.00705277 0.00743053 0.00777798 0.00809547
 0.00838408 0.0086453  0.00888088 0.00909271 0.00928271 0.00945275
 0.00960466 0.00974015 0.00986084 0.00996822 0.01006366 0.01014841
 0.01022362 0.01029031 0.0103494  0.01040175 0.0104481  0.01048912
 0.01052541 0.01055751 0.0105859  0.01061099 0.01063316]
[       nan 0.04637371 0.05511857 0.06129149 0.06600252 0.06972956
 0.07274117 0.07520941 0.07725339 0.07895968 0.08039338 0.08160464
 0.08263276 0.08350901 0.08425854 0.08490172 0.08545526 0.08593288
 0.08634596 0.08670398 0.08701487 0.0872853  0.0875209  0.08772645
 0.08790599 0.088063   0.08820045 0.08832087 0.08842647 0.08851913
 0.08860049 0.08867198 0.08873481 0.08879007 0.08883869 0.08888147
 0.08891914 0.0889523  0.08898152 0.08900726 0.08902994]


Input: Same as above, except with the following included:

\begin{equation*}
\begin{aligned}
\text{size\_ci}&=\text{size of confidenceinterval, i.e. 0.95 as 95\% confidence interval} \\
\text{method}&=\text{for either one or two-sided confidence interval}
\end{aligned}
\end{equation*}

Output: Upper and lower confidence interval of the short rate

Formula:

\begin{equation*}
\begin{aligned}
\text{mean}&=r_{0}\cdot\exp\left(-a\cdot(T-t)\right)+\frac{b}{a}\cdot\left(1-\exp\left[-a\cdot(T-t)\right]\right) \\
\text{std.}&=\sqrt{\frac{\sigma^{2}}{2\cdot a}\cdot\left(1-\exp\left[-a\cdot(T-t)\right]\right)}
\end{aligned}
\end{equation*}

Of course, the confidence intervals are drawn from a normal distribution and calculated as $\text{mean}=\text{std.}\cdot\text{quantile}(0.95,\mathcal{N}(0,1))$

The code is horrible written and basically repeats itself six times

#### Euro option price for vasicek

In [ ]:
def euro_option_price_vasicek(K,T,U,p_T,p_U,a,sigma,type = "call"):
    sigma_p = (sigma/a)*(1-np.exp(-a*(U-T)))*np.sqrt((1-np.exp(-2*a*T))/(2*a))
    d1 = (np.log(p_U/(p_T*K)))/sigma_p + 0.5*sigma_p
    d2 = d1 - sigma_p
    if type == "call":
        price = p_U*ndtr(d1) - p_T*K*ndtr(d2)
    elif type == "put":
        price = p_T*K*ndtr(-d2) - p_U*ndtr(-d1)
    return price

#Example: Not right now

Calculate the following (from slides):
"We would like to price a European call optioon with strike $K$ and maturity $T_1$ on an underlying zero coupon bond with maturity $T_2$.

In the notation of the previous discussion, we have that $T=T_1$, $S(t)=p(t,T_2)$ and must use $p(t,T_1)$ as numeraire" 

Input:

\begin{equation*}
\begin{aligned}
K&=\text{strike price} \\
U&=T_{1} \\
T&=T_{2} \\
p_{T}&=p(0,T_{2}) \\
p_{U}&=p(0,T_{1})
\end{aligned}
\end{equation*}

Output: The price of the European call option

Formula: Using the notation from slides and rewriting, we get:

\begin{equation*}
\begin{aligned}
\text{Call: }\Pi(0;\chi)&=p(0,T_{2})\Phi(d_{1})-Kp(0,T_{1})\Phi(d_{2}) \\

\text{Put: }\Pi(0;\chi)&=p(0,T_{1})K\Phi(-d_{2})-p(0,T_{2})\Phi(-d_{1}) \\

d_{1} &=\frac{\ln\left(\frac{p(0,T_{2})}{Kp(0,T_{1})}\right)+\frac{1}{2}\Sigma^{2}}{\sqrt{\Sigma^{2}}}=\frac{\ln\left(\frac{p(0,T_{2})}{Kp(0,T_{1})}\right)}{\Sigma}+\frac{1}{2}\Sigma\quad\text{and}\quad d_{2}=d_{1}-\sqrt{\Sigma^{2}}=d_{1}-\Sigma \\

\Sigma^{2}&=\frac{\sigma^{2}}{2a^{3}}\left[1-e^{2aT_{1}}\right]\cdot\left[1-e^{-a(T_{2}-T_{1})}\right]^{2}\Leftrightarrow \\

\Sigma &=\frac{\sigma}{a}\cdot\left[1-e^{-a(T_{2}-T_{1})}\right]\sqrt{\frac{1}{2a}\left[1-e^{2aT_{1}}\right]}
\end{aligned}
\end{equation*}

Here, $\Phi$ is the cumulative distribution of the standard normal distribution, and we use the scipy Python function 'ndtr' to calculate this.

#### Fit vasicek object (calculate squared errors)

In [ ]:
def fit_vasicek_obj(param,R_star,T,scaling = 1):
    r0, a, b, sigma = param
    M = len(T)
    R_fit = spot_rate_vasicek(r0,a,b,sigma,T)
    y = 0
    for m in range(0,M):
        y += scaling*(R_fit[m] - R_star[m])**2
    return y

The functions simply returns the squared errors of between a fitted model and actual data. It is used to optimize the parameters of our vasicek model so it fits to real data.

Input:

\begin{equation*}
\begin{aligned}
\text{param}&=\text{Parameters that we want to optimize over} \\

\text{R\_star}&=\text{Actual data we need to fit our model over} \\

T&=\text{Time to maturity} \\

\text{scaling}&=\text{We might want to scale our data if errors are too small or too large for the optimizer}
\end{aligned}
\end{equation*}

Output: Squared errors (scaled if needed)

In [ ]:
def fit_vasicek_no_sigma_obj(param,sigma,R_star,T,scaling = 1):
    r0, a, b = param
    M = len(T)
    R_fit = spot_rate_vasicek(r0,a,b,sigma,T)
    y = 0
    for m in range(0,M):
        y += scaling*(R_fit[m] - R_star[m])**2
    return y

Same as above, just without $\sigma$

#### ZCB for Cox-Ingersoll-Ross model
The following models are essentially the same as above, and I will therefore only provide the formulas. Remember that the CIR model has the following dynamics:

\begin{equation*}
dr_{t}=(b-ar_{t})dt+\sigma\sqrt{r_{t}}dB_{t}
\end{equation*}

In [ ]:
def zcb_price_cir(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float or type(tau) == np.float64:
        gamma = np.sqrt(a**2+2*sigma**2)
        D = (gamma+a)*(np.exp(gamma*tau)-1)+2*gamma
        A = ((2*gamma*np.exp(0.5*tau*(a+gamma)))/D)**((2*a*b)/(sigma**2))
        B = 2*(np.exp(gamma*tau)-1)/D
        p = A*np.exp(-r0*B)
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        p = np.zeros([M])
        for i in range(0,M):
            gamma = np.sqrt(a**2+2*sigma**2)
            D = (gamma+a)*(np.exp(gamma*tau[i])-1)+2*gamma
            A = ((2*gamma*np.exp(0.5*tau[i]*(a+gamma)))/D)**((2*a*b)/(sigma**2))
            B = 2*(np.exp(gamma*tau[i])-1)/D
            p[i] = A*np.exp(-r0*B)
    else:
        print(f"tau not a recognized type")
        p = False
    return p

Formula:

\begin{equation*}
\begin{aligned}
p(t,T)&=A_{0}(T-t)e^{-B(T-t)r} \\
\end{aligned}
\end{equation*}

where

\begin{equation*}
\begin{aligned}
A_{0}(T-t)&=\left[\frac{2\gamma\exp\left(\frac{(T-t)(a+\gamma)}{2}\right)}{D}\right]^{\frac{2ab}{\sigma^{2}}} \\

B(T-t)&=\frac{2\left(\exp[\gamma(T-t)]-1\right)}{D} \\

D&=(\gamma+a)\left(\exp[\gamma(T-t)]-1\right)+2\gamma \\
\gamma&=\sqrt{a^{2}+2\sigma^{2}}
\end{aligned}
\end{equation*}


#### Spot rate for Cox-Ingersoll-Ross model

In [ ]:
def spot_rate_cir(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        if tau < 1e-6:
            r = 0
        else:
            gamma = np.sqrt(a**2+2*sigma**2)
            D = (gamma+a)*(np.exp(gamma*tau)-1)+2*gamma
            A = ((2*gamma*np.exp(0.5*tau*(a+gamma)))/D)**((2*a*b)/(sigma**2))
            B = 2*(np.exp(gamma*tau)-1)/D
            r = (-np.log(A)+r0*B)/(tau)
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        M = len(tau)
        r = np.zeros([M])
        for i in range(0,M):
            if tau[i] < 1e-6:
                r[i] = 0
            else:
                gamma = np.sqrt(a**2+2*sigma**2)
                D = (gamma+a)*(np.exp(gamma*tau[i])-1)+2*gamma
                A = ((2*gamma*np.exp(0.5*tau[i]*(a+gamma)))/D)**((2*a*b)/(sigma**2))
                B = 2*(np.exp(gamma*tau[i])-1)/D
                r[i] = (-np.log(A)+r0*B)/(tau[i])
    else:
        print(f"tau not a recognized type")
        r = False
    return r

Formula:

\begin{equation*}
R(t,T)=\frac{-\log A_{0}+rB}{T-t}
\end{equation*}

where $A_{0}$ and $B$ are the same as above.

#### Forward rate for Cox-Ingersoll-Ross model

In [ ]:
def forward_rate_cir(r0,a,b,sigma,tau):
    if type(tau) == int or type(tau) == float:
        if tau < 1e-6:
            f = 0
        else:
            c = (2*a*b)/(sigma**2)
            gamma = np.sqrt(a**2+2*sigma**2)
            N = 2*gamma*np.exp(0.5*tau*(a+gamma))
            N_T = gamma*(gamma+a)*np.exp(0.5*tau*(a+gamma))
            D = (gamma+a)*(np.exp(gamma*tau)-1)+2*gamma
            D_T = gamma*(a+gamma)*np.exp(gamma*tau)
            M = 2*(np.exp(gamma*tau)-1)
            M_T = 2*gamma*np.exp(gamma*tau)
            f = c*(-N_T/N+D_T/D)+r0*(M_T*D-M*D_T)/D**2
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        N = len(tau)
        f = np.zeros([N])
        for i in range(0,N):
            if tau[i] < 1e-6:
                f[i] = 0
            else:
                c = (2*a*b)/(sigma**2)
                gamma = np.sqrt(a**2+2*sigma**2)
                N = 2*gamma*np.exp(0.5*tau[i]*(a+gamma))
                N_T = gamma*(gamma+a)*np.exp(0.5*tau[i]*(a+gamma))
                D = (gamma+a)*(np.exp(gamma*tau[i])-1)+2*gamma
                D_T = gamma*(a+gamma)*np.exp(gamma*tau[i])
                M = 2*(np.exp(gamma*tau[i])-1)
                M_T = 2*gamma*np.exp(gamma*tau[i])
                f[i] = c*(-N_T/N+D_T/D)+r0*(M_T*D-M*D_T)/D**2
    else:
        print(f"tau not a recognized type")
        f = False
    return f

Formula:

\begin{equation*}
\begin{aligned}
f(t,T)=c\left(-\frac{N_{t}}{N}+\frac{D_{t}}{D}\right)+rB_{t}
\end{aligned}
\end{equation*}

where

\begin{equation*}
\begin{aligned}
N_{t}&=\gamma(a+\gamma)\cdot\exp\left(\frac{(T-t)(a+\gamma)}{2}\right) \\

N&=2\gamma\cdot\exp\left(\frac{(T-t)(a+\gamma)}{2}\right) \\

D_{t}&=\gamma(\gamma+a)\cdot\exp(\gamma(T-t)) \\

D&=(\gamma+a)\left(\exp[\gamma(T-t)]-1\right)+2\gamma \\

B_{T}&=\frac{M_{T}D-MD_{T}}{D^{2}} \\

c&=\frac{2ab}{\sigma^{2}} \\
\end{aligned}
\end{equation*}

Niether $c$ or $N$ are defined in the slides.

#### Confidence intervals for Cox-Ingersoll-Ross

In [ ]:
def ci_cir(r0,a,b,sigma,tau,size_ci,method = "two_sided"):
    if type(tau) == int or type(tau) == float:
        if method == "lower":
            if tau < 1e-6:
                lb, ub = np.nan, np.nan
            else:
                df = (4*a*b)/sigma**2
                nc = (4*a*np.exp(-a*tau)*r0)/(sigma**2*(1-np.exp(-a*tau)))
                scaling = (4*a)/(sigma**2*(1-np.exp(-a*tau)))
                lb, ub = ncx2.ppf(1-size_ci,df,nc)/scaling, np.inf
        elif method == "upper":
            if tau < 1e-6:
                lb, ub = np.nan, np.nan
            else:
                df = (4*a*b)/sigma**2
                nc = (4*a*np.exp(-a*tau)*r0)/(sigma**2*(1-np.exp(-a*tau)))
                scaling = (4*a)/(sigma**2*(1-np.exp(-a*tau)))
                lb, ub = 0, ncx2.ppf(size_ci,df,nc)/scaling
        elif method == "two_sided":
            if tau < 1e-6:
                lb, ub = np.nan, np.nan
            else:
                df = (4*a*b)/sigma**2
                nc = (4*a*np.exp(-a*tau)*r0)/(sigma**2*(1-np.exp(-a*tau)))
                scaling = (4*a)/(sigma**2*(1-np.exp(-a*tau)))
                lb, ub = ncx2.ppf((1-size_ci)/2,df,nc)/scaling, ncx2.ppf(size_ci+(1-size_ci)/2,df,nc)/scaling
    elif type(tau) == tuple or type(tau) == list or type(tau) == np.ndarray:
        N = len(tau)
        lb, ub = np.zeros([N]), np.zeros([N])
        if method == "lower":
            for i in range(0,N):
                if tau[i] < 1e-6:
                    lb[i], ub[i] = np.nan, np.nan
                else:
                    df = (4*a*b)/sigma**2
                    nc = (4*a*np.exp(-a*tau[i])*r0)/(sigma**2*(1-np.exp(-a*tau[i])))
                    scaling = (4*a)/(sigma**2*(1-np.exp(-a*tau[i])))
                    lb[i], ub[i] = ncx2.ppf(1-size_ci,df,nc)/scaling, np.inf
        elif method == "upper":
            for i in range(0,N):
                if tau[i] < 1e-6:
                    lb[i], ub[i] = np.nan, np.nan
                else:
                    df = (4*a*b)/sigma**2
                    nc = (4*a*np.exp(-a*tau[i])*r0)/(sigma**2*(1-np.exp(-a*tau[i])))
                    scaling = (4*a)/(sigma**2*(1-np.exp(-a*tau[i])))
                    lb[i], ub[i] = 0, ncx2.ppf(size_ci,df,nc)/scaling
        elif method == "two_sided":
            for i in range(0,N):
                if tau[i] < 1e-6:
                    lb[i], ub[i] = np.nan, np.nan
                else:
                    df = (4*a*b)/sigma**2
                    nc = (4*a*np.exp(-a*tau[i])*r0)/(sigma**2*(1-np.exp(-a*tau[i])))
                    scaling = (4*a)/(sigma**2*(1-np.exp(-a*tau[i])))
                    lb[i], ub[i] = ncx2.ppf((1-size_ci)/2,df,nc)/scaling, ncx2.ppf(size_ci+(1-size_ci)/2,df,nc)/scaling
                    # (4*b)/(sigma**2*(1-np.exp(-b*tau[i])))*
    else:
        print(f"tau not a recognized type")
        lb,ub = False, False
    return lb, ub

To calculate the confidence intervals for the CIR model, we use a noncentral chi-squared distribution (it's easy to see...). 

\begin{equation*}
\begin{aligned}
\text{df}&=\frac{4ab}{\sigma^{2}} \\

\text{nc}&=\frac{4a\exp(-a(T-t)r)}{\sigma^{2}(1-\exp[-a(T-t)])} \\

\text{scaling}&=\frac{4a}{\sigma^{2}(1-\exp[-a(T-t)])}
\end{aligned}
\end{equation*}

Now we can calculate the upper and lower bounds using the Percent point function (inverse of cdf — percentiles):

\begin{equation*}
lb=\frac{Q(\frac{1-0.95}{2},\text{df},\text{nc})}{\text{scaling}},ub=\frac{Q(0.95+\frac{1-0.95}{2},\text{df},\text{nc})}{\text{scaling}}
\end{equation*}

#### Fit Cox-Ingersoll-Ross object (calculate squared errors)
See the fit for vasicek. The procedure is exactly the same.

In [ ]:
def fit_cir_obj(param,R_star,T,scaling = 1):
    r0, a, b, sigma = param
    M = len(T)
    R_fit = spot_rate_cir(r0,a,b,sigma,T)
    y = 0
    for m in range(0,M):
        y += scaling*(R_fit[m] - R_star[m])**2
    return y

def fit_cir_no_sigma_obj(param,sigma,R_star,T,scaling = 1):
    r0, a, b = param
    M = len(T)
    R_fit = spot_rate_cir(r0,a,b,sigma,T)
    y = 0
    for m in range(0,M):
        y += scaling*(R_fit[m] - R_star[m])**2
    return y

#### Simulation of the short rate

In [ ]:

def short_rate_simul(r0,param,M,T,method = "vasicek"):
    r = np.zeros([M+1])
    r[0] = r0
    delta = T/M
    if method == "vasicek":
        a, b, sigma = param
        delta_sqrt = np.sqrt(delta)
        Z = np.random.standard_normal(M)
        for m in range(1,M+1):
            r[m] = r[m-1] + (b-a*r[m-1])*delta + sigma*delta_sqrt*Z[m-1]
    elif method == "cir":
        a, b, sigma = param
        Z = np.random.standard_normal(M)
        for m in range(1,M+1):
            r[m] = r[m-1] + a*(b-r[m-1])*delta + sigma*np.sqrt(delta*r[m-1])*Z[m-1]
    elif method == "ho_lee_ns":
        f_inf, a, b, sigma = param
        Z = np.random.standard_normal(M)
        for m in range(1,M+1):
            param_theta = (f_inf, a, b, sigma)
            theta = theta_ns(param_theta,m*delta)
            r[m] = r[m-1] + theta*delta + sigma*np.sqrt(delta*r[m-1])*Z[m-1]
    return r

#### Spot rate bump

In [ ]:
def spot_rate_bump(T_bump,size_bump,T,R_input,p_input):
    R, p = R_input.copy(), p_input.copy()
    if type(T_bump) == int or type(T_bump) == float or type(T_bump) == np.float64:
        I_bump, idx_bump = value_in_list_returns_I_idx(T_bump,T)
        R[idx_bump] = R[idx_bump] + size_bump
        p[idx_bump] = np.exp(-R[idx_bump]*T_bump)
    elif type(T_bump) == tuple or type(T_bump) == list or type(T_bump) == np.ndarray:
        if type(size_bump) == int or type(size_bump) == float or type(size_bump) == np.float64:
            for i in range(0,len(T_bump)):
                I_bump, idx_bump = value_in_list_returns_I_idx(T_bump[i],T)
                R[idx_bump] = R[idx_bump] + size_bump
                p[idx_bump] = np.exp(-R[idx_bump]*T_bump[i])
        elif type(size_bump) == tuple or type(size_bump) == list or type(size_bump) == np.ndarray:
            for i in range(0,len(T_bump)):
                I_bump, idx_bump = value_in_list_returns_I_idx(T_bump[i],T)
                R[idx_bump] = R[idx_bump] + size_bump[i]
                p[idx_bump] = np.exp(-R[idx_bump]*T_bump[i])
    return R, p


